In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

#과제

##Titanic 데이터를 가지고 5 개의 FE 방법들 직접 사용해보기 (오늘 피피티에 소개되지 않은 FE 방법도 괜찮음)

##0. 데이터 확인

In [3]:
titanic = pd.read_csv('/content/drive/MyDrive/YBIGTA/0719 FE/titanic.csv')
titanic.info()
titanic.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


##1. 결측값 대체 

In [4]:
# 결측값 찾기
print(titanic.isnull().sum())
print('-----------')
# 결측값이 너무 많은 Cabin은 Drop (총 Column 418개 중 327개가 NaN, 78%가 결측값)
titanic = titanic.drop('Cabin', axis=1)

# Age의 최빈값 찾기 
titanic['Age'].value_counts()

# Age는 최빈값으로 대체 
titanic.loc[titanic['Age'] != titanic['Age'], 'Age'] = 21

# Fare는 평균값으로 대체
titanic.loc[titanic['Fare'] != titanic['Fare'], 'Fare'] = titanic.Fare.mean()

# 확인
print(titanic.isnull().sum())
print('-----------')

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64
-----------
PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64
-----------


##2. 지표변수

In [5]:
# 나이 범위 확인
print(f'최소나이 : {titanic.Age.min()}\n최대나이 : {titanic.Age.max()}')

# 지표 생성
human = []
for a in titanic['Age']:
  if 0 <= a < 1:
    human.append('Baby')
  elif 1<= a < 10:
    human.append('Child')
  elif 10<= a < 20:
    human.append('Teenager')
  elif 20<= a <30:
    human.append('Youth')
  elif 30<= a < 45:
    human.append('Mature')
  else:
    human.append('Elder')

# 지표 추가
titanic['Human_development'] = human

# 확인
titanic.head(10)

최소나이 : 0.17
최대나이 : 76.0


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Human_development
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q,Mature
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S,Elder
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q,Elder
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S,Youth
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S,Youth
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,S,Teenager
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,Q,Mature
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,S,Youth
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,C,Teenager
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,S,Youth


##3. Feature Split

In [6]:
# 이름의 First, Title, Second 로 Split 진행

# Split
first = []
temp = []
title = []
second = []

for n in titanic['Name'].str.split(','):
  first.append(n[0])
  temp.append(n[1])

for word in temp:
  m = word.split('.')
  title.append(m[0])
  second.append(m[1])

# 대입
titanic['First_name'] = first
titanic['Title_name'] = title
titanic['Second_name'] = second

# 확인
titanic.head(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Human_development,First_name,Title_name,Second_name
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q,Mature,Kelly,Mr,James
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S,Elder,Wilkes,Mrs,James (Ellen Needs)
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q,Elder,Myles,Mr,Thomas Francis
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S,Youth,Wirz,Mr,Albert
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S,Youth,Hirvonen,Mrs,Alexander (Helga E Lindqvist)
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,S,Teenager,Svensson,Mr,Johan Cervin
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,Q,Mature,Connolly,Miss,Kate
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,S,Youth,Caldwell,Mr,Albert Francis
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,C,Teenager,Abrahim,Mrs,Joseph (Sophie Halaut Easu)
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,S,Youth,Davies,Mr,John Samuel


##4. Scaling

In [7]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
mm_scaler = MinMaxScaler()
sd_scaler = StandardScaler()

# Fare scaling
mm_scaler = mm_scaler.fit_transform(titanic['Fare'].values.reshape(-1,1))
titanic['mm_scaled_fare'] = mm_scaler

# Age scaling
sd_scaler = sd_scaler.fit_transform(np.log1p(titanic['Age']).values.reshape(-1,1))
titanic['sd_scaled_age'] = sd_scaler

# 확인
titanic.head(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Human_development,First_name,Title_name,Second_name,mm_scaled_fare,sd_scaled_age
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q,Mature,Kelly,Mr,James,0.015282,0.542345
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S,Elder,Wilkes,Mrs,James (Ellen Needs),0.013663,1.061671
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q,Elder,Myles,Mr,Thomas Francis,0.018909,1.529809
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S,Youth,Wirz,Mr,Albert,0.016908,0.133782
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S,Youth,Hirvonen,Mrs,Alexander (Helga E Lindqvist),0.023984,-0.204858
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,S,Teenager,Svensson,Mr,Johan Cervin,0.018006,-0.940708
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,Q,Mature,Connolly,Miss,Kate,0.014891,0.309002
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,S,Youth,Caldwell,Mr,Albert Francis,0.056604,0.071174
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,C,Teenager,Abrahim,Mrs,Joseph (Sophie Halaut Easu),0.014110,-0.533762
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,S,Youth,Davies,Mr,John Samuel,0.047138,-0.281382


##5. One-Hot Encoding

In [8]:
# Pclass 인코딩
pclass = pd.get_dummies(titanic['Pclass'])
pclass.columns= ['Pclass_1', 'Pclass_2', 'Pclass_3']
# Emarked 인코딩
embarked =pd.get_dummies(titanic['Embarked'])

# 대입
encoded = pd.concat([pclass, embarked], axis=1)
titanic_final = pd.concat([titanic, encoded], axis=1)

# 확인
titanic_final.head(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,...,Title_name,Second_name,mm_scaled_fare,sd_scaled_age,Pclass_1,Pclass_2,Pclass_3,C,Q,S
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q,...,Mr,James,0.015282,0.542345,0,0,1,0,1,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S,...,Mrs,James (Ellen Needs),0.013663,1.061671,0,0,1,0,0,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q,...,Mr,Thomas Francis,0.018909,1.529809,0,1,0,0,1,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S,...,Mr,Albert,0.016908,0.133782,0,0,1,0,0,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S,...,Mrs,Alexander (Helga E Lindqvist),0.023984,-0.204858,0,0,1,0,0,1
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,S,...,Mr,Johan Cervin,0.018006,-0.940708,0,0,1,0,0,1
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,Q,...,Miss,Kate,0.014891,0.309002,0,0,1,0,1,0
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,S,...,Mr,Albert Francis,0.056604,0.071174,0,1,0,0,0,1
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,C,...,Mrs,Joseph (Sophie Halaut Easu),0.014110,-0.533762,0,0,1,1,0,0
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,S,...,Mr,John Samuel,0.047138,-0.281382,0,0,1,0,0,1
